In [1]:
import pandas as pd
import os
import json
import re
from bs4 import BeautifulSoup

### House Address ,City ,ZipCode Information

In [2]:
# same as in group data
def get_house_details(content,realestate_df,index):
    
    #Price
    #price = soup.find(class_='info-block price').find(class_="statsValue").find_all('span')[1].get_text()
    # realestate_df.loc[index]['Price'] = price
    
    #Street Address
    street_Addr = soup.find(class_='street-address').get_text()
    realestate_df.loc[index]['Address'] = street_Addr
    
    #City
    city = soup.find(class_='locality').get_text()[:-2]
    realestate_df.loc[index]['City'] = city 
    
    #State
    state = soup.find(class_='region').get_text()
    realestate_df.loc[index]['State'] = state
    
    #Zipcode
    zipcode = soup.find(class_='postal-code').get_text()
    realestate_df.loc[index]['ZipCode'] = zipcode

### Get Detail House Information

In [3]:
# same as in main group data
#get bed and bath information
def bedsDetail(content):
    noOfBeds = soup.findAll("div", attrs = {'class': 'info-block','data-rf-test-id' : 'abp-beds'})
    noOfBedsList = []
    for value in noOfBeds:
        noOfBedsList.append(value.get_text())
    
    noOfBedsList2 = []
    for value2 in noOfBedsList:
        noOfBedsList2.append(value2[0:1])
    return noOfBedsList2

def bathsDetail(content):
    noOfBaths = soup.findAll("div", attrs = {'class': 'info-block','data-rf-test-id' : 'abp-baths'})
    noOfBathsList = []
    for value in noOfBaths:
        noOfBathsList.append(value.get_text())
    
    noOfBathsList2 = []
    for value2 in noOfBathsList:
        noOfBathsList2.append(value2[0:1])
    return noOfBathsList2

In [4]:
# same as in group data
# Property Type
def PropertyDetail(content):
    PropertyType = content.findAll('div',attrs={'class':'facts-table'})
    PropertyType=PropertyType[0].contents[7].contents[1].get_text()
    dct_PropertyDetail = {}
    dct_PropertyDetail['PropertyType']=PropertyType
    return dct_PropertyDetail 

### get Sale History Events

In [5]:
def SaleEvents(content):
    html = content.get_text()
    matches = re.findall('events\\\\":([^]]*)',html)
    string = str(matches[0] + ']')
    string = string.replace('\\\"','\"')
    events = json.loads(string)
    saleHistory = []
    yearBuilt=BuiltDetail(content)['YearBuilt']
    prevSalePrice = 0
    for event in events:
        if 'price' not in event:
            continue
        if event['price'] == prevSalePrice:
            continue
        if 'eventDescription' not in event:
            continue
        if not event['eventDescription'].startswith('Sold'):
            continue
        if 'eventDateString' not in event:
            continue
        saleYear = event['eventDateString'].split(',')[1].strip()
        if saleYear == yearBuilt:
            continue
        prevSalePrice = event['price']
        #print('Event: ' + event['eventDescription'] + ' Price: ' + str(event['price']) + ' SaleYear: ' + saleYear)
        saleHistory.append({'SaleYear':saleYear,'SalePrice':event['price'], 'YearsOwned':0})
    for i in range(len(saleHistory)):
        #print(saleHistory)
        saleYear = int(saleHistory[i]['SaleYear'])
        if i == (len(saleHistory)-1):
            saleHistory[i]['YearsOwned'] = saleYear - int(yearBuilt)
        else:
            saleHistory[i]['YearsOwned'] = saleYear - int(saleHistory[i+1]['SaleYear'])
    #print(saleHistory)
    return saleHistory

### Built Year and Renovate Year

In [6]:
#same as in group data
# Built Year and Renovated Year
def BuiltDetail(content):
    Built = content.findAll('div',attrs={'class':'facts-table'})
    BuiltYear=Built[0].contents[8].contents[1].get_text()
    RenovatedYear=Built[0].contents[9].contents[1].get_text()
    dct_BuiltDetail = {}
    dct_BuiltDetail['YearBuilt']=BuiltYear
    dct_BuiltDetail['YearRenovated']=RenovatedYear
    return dct_BuiltDetail   

### Populate Built Year and Renovate year to DataFrame

In [7]:
# same as in group data
# Built year and Renovated Details
def write_Built_toDataFrame(content,realestate_df,index):
    try:
        dct_BuiltDetail=BuiltDetail(content)
        for key, value in dct_BuiltDetail.items() :
            #print (key, value)
            if 'YearBuilt' in key:
                realestate_df.loc[index]['YearBuilt'] = value
            elif 'YearRenovated' in key:
                realestate_df.loc[index]['YearRenovated'] = value
    except Exception as e:
        print(" error at year built and Renovation :", e)
        

In [8]:
# same as in group data
#Property_Type Detail to DataFrame        
# Property Type

def write_PropertyType_toDataFrame(content,realestate_df,index):
    try:
        dct_PropertyDetail=PropertyDetail(content)
        for key, value in dct_PropertyDetail.items() :
            #print (key, value)
            if 'PropertyType' in key:
                realestate_df.loc[index]['PropertyType'] = value
    except Exception as e:
        print("error while writing property info:",e)     

In [9]:

#same as in group data
def write_noOfBeds_toDataFrame(content,realestate_df,index):
    try:
        beds_List = bedsDetail(content)
        for value in beds_List:
            #print(value)
            realestate_df.loc[index]['Beds'] = value
    except Exception as e:
        print("error while writing bedroom info:",e)    
        
def write_noOfBaths_toDataFrame(content,realestate_df,index):
    try:
        baths_List = bathsDetail(content)
        for value in baths_List:
            #print(value)
            realestate_df.loc[index]['Baths'] = value
    except Exception as e:
        print("error while writing bathroom info:",e)

In [10]:
# same as in group data
#selling Price to dataFrame
def write_sellingPrice_toDataFrame(content, realestate_df, index):
    try:
        sellingPriceList = sellingPrice(content)

        for value in sellingPriceList:
            #print (value)
            realestate_df.loc[index]['Price'] = value
    except Exception as e:
               
        print("Exception in write sellingPrice_toDataFrame :", e)
        

In [11]:
#same as in group data
#get Selling Price
def sellingPrice(content):
    sellingPrice = soup.findAll("div", attrs = {"class": "statsValue"})


    sellingPriceList = []
    sellingPriceList.append(sellingPrice[0].get_text())
    
    
    
    return sellingPriceList

In [12]:
def write_saleHistory_toDataFrame(saleHistory, realestate_df,index):
    try:
        realestate_df.loc[index]['SaleYear']=saleHistory['SaleYear']
        realestate_df.loc[index]['SalePrice']=saleHistory['SalePrice']
        realestate_df.loc[index]['YearsOwned']=saleHistory['YearsOwned']
    except Exception as e:
        print('error while writing sale history info:',e)

## Main function start here

In [13]:

# main function starts here
# Reading html file from the directory using BeautifulSoup


column_name =['PropertyType','Address','City','State','ZipCode','Price','Beds','Baths',
             'YearBuilt','YearRenovated','SaleYear','SalePrice','YearsOwned']
realestate_df = pd.DataFrame(columns =column_name)
counter=0
os.chdir('C:/Users/Abhisha Burande/Downloads/html2/html/all')

for file in os.listdir():
    try:
        print("reading file" ,file)
        soup = BeautifulSoup(open(file,encoding='utf-8'), "html.parser")
        saleHistory = SaleEvents(soup)
        for i in range(len(saleHistory)):
            house_index ="house_"+str(counter)+'.'+str(i)
            realestate_df= realestate_df.append(pd.Series(name=house_index))
            #print('0')
            write_saleHistory_toDataFrame(saleHistory[i],realestate_df,house_index)
            #print("1")
            write_noOfBeds_toDataFrame(soup,realestate_df,house_index)
            #print("2")
            write_noOfBaths_toDataFrame(soup,realestate_df,house_index)
            #print("3")
            write_Built_toDataFrame(soup,realestate_df,house_index)
            #print("4")
            write_PropertyType_toDataFrame(soup,realestate_df,house_index)
            #print("5")
            get_house_details(soup,realestate_df,house_index)
            #print("6")
            write_sellingPrice_toDataFrame(soup,realestate_df,house_index)
    except Exception as e:
        print("error in File :",file)
        print(e)
    except:
        print(file)
    finally:
        counter=counter+1

reading file CA_Campbell_123-E-Rosemary-Ln-95008_home_1738332.html
[{'isEventAdminOnly': False, 'price': 1250000, 'isPriceAdminOnly': False, 'eventDescription': 'Listed', 'mlsDescription': 'Active', 'source': 'MLSListings', 'sourceId': 'ML81784293', 'dataSourceDisplay': {'dataSourceId': 8, 'dataSourceDescription': 'MLSListings Inc. (MLSListings)', 'dataSourceName': 'MLSListings', 'dataSourceImage': 'mlsListings_small.png', 'shouldShowLargerLogo': False}, 'priceDisplayLevel': 1, 'historyEventType': 1, 'eventDate': 1582764833000}, {'isEventAdminOnly': False, 'price': 690000, 'isPriceAdminOnly': False, 'eventDescription': 'Sold (MLS)', 'mlsDescription': 'Sold', 'source': 'MLSListings', 'sourceId': 'ML81303406', 'dataSourceDisplay': {'dataSourceId': 8, 'dataSourceDescription': 'MLSListings Inc. (MLSListings)', 'dataSourceName': 'MLSListings', 'dataSourceImage': 'mlsListings_small.png', 'shouldShowLargerLogo': False}, 'priceDisplayLevel': 1, 'historyEventType': 2, 'eventDate': 1362643200000

In [14]:
realestate_df

,PropertyType,Address,City,State,ZipCode,Price,Beds,Baths,YearBuilt,YearRenovated,SaleYear,SalePrice,YearsOwned
house_0.0,Single Family Residential,123 E Rosemary Ln,CAMPBELL,CA,95008,"$1,250,000",3,2,1949,1949,2013,690000,1
house_0.1,Single Family Residential,123 E Rosemary Ln,CAMPBELL,CA,95008,"$1,250,000",3,2,1949,1949,2012,510000,63
house_2.0,Single Family Residential,673 Regas Dr,CAMPBELL,CA,95008,"$1,989,888",4,3,1988,1988,2002,967000,14


In [65]:
realestate_df.to_csv('realestate_sale_history_all_df.csv')